In [1]:
# GodQuestions_Crawling

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from datetime import date

## 2025/04/30

In [9]:
final=pd.DataFrame(columns=["crawling_date","big_title_far","Question_FAR","Answer_FAR","URL_FAR","Question_ENG","Answer_ENG","URL_ENG"])
print(final)

Empty DataFrame
Columns: [crawling_date, big_title_far, Question_FAR, Answer_FAR, URL_FAR, Question_ENG, Answer_ENG, URL_ENG]
Index: []


In [10]:
# 1) main title -> click & title extracting
driver = webdriver.Chrome()
driver.get("https://www.gotquestions.org/Farsi/")

a_tags=driver.find_elements(By.TAG_NAME,"a")
print(len(a_tags))

52


In [11]:
for i in range(1, 52): # 첫 번째 클릭 (대주제) - 1부터 52까지
    

    try:
        element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{i}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
        big_title_far = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{i}]').text # 대주제도 데이터 프레임에 넣기
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)

    except NoSuchElementException:
            time.sleep(0.5)
            break

    for ii in range(1, 120): # 두 번째 클릭 (소주제) 

        # 소주제 개수는 대주제마다 달라서, range()로 일단 1부터 120까지 범위 정해놓고
        # 반복문을 실행할 수 없는 상황, 찾으려는 요소가 없어서 더 이상 작업할 수 없을 때 반복문 종료
        try:
            element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{ii}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
            driver.execute_script("arguments[0].click();", element)
            time.sleep(0.5)
        
        # 만약 버튼이 없다면 초기 화면으로 돌아간 후 초기 반복문으로 이동
        except NoSuchElementException:
            driver.get("https://www.gotquestions.org/Farsi/")
            time.sleep(0.5)
            break
            
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # crawling 하는 날짜 date 넣기
        crawling_date = date.today()
        # 제목(소주제)는 <h1></h1> 사이에 존재
        title_far = soup.find('h1').get_text(strip = True)

        # 현재 url
        url_far = driver.current_url
            
        # 답변 내용은 "جواب"과 "English"사이에 존재한다는 공통점
        content_str_far = str(soup.get_text(separator="\n"))
        start_far = content_str_far.find('جواب') + len('جواب') # 답변의 시작 지점
        end_far = content_str_far.find('English') # 답변의 끝 지점
        answer_html_far = content_str_far[start_far : end_far]
        answer_far = BeautifulSoup(answer_html_far, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기

        # ----------- #

        # English 클릭해서
        #element = driver.find_element(By.XPATH,"/html/body/main/section[1]/div/a")
        
        try :
            element = driver.find_element(By.LINK_TEXT, "English")
            driver.execute_script("arguments[0].click();", element)
            time.sleep(0.5)

        # English 없는 경우 발생...ㅎㄷㄷ
        except NoSuchElementException: 
             # 페르시아어 파일 저장
            new_row = pd.DataFrame({"crawling_date":[crawling_date], "big_title_far":[big_title_far],
                                    "Question_FAR": [title_far], "Answer_FAR": [answer_far], "URL_FAR": [url_far],
                                    "Question_ENG": "NA", "Answer_ENG": "NA", "URL_ENG": "NA"})
            final = pd.concat([final, new_row], ignore_index=True)
            driver.back()
            time.sleep(0.1)
            break

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
                
        # 영어 내용 크롤링
        # 영어 제목(소주제) <h1></h1> 사이에 존재
        title_eng = soup.find('h1').get_text(strip = True)

        # 현재 url
        url_eng = driver.current_url
                
        # 답변 내용은 "Question"과 "Return to:" 사이에 존재한다는 공통점
        content_str_eng = str(soup.get_text(separator="\n"))
        start_eng = content_str_eng.find('Answer') + len('Answer') # 답변의 시작 지점
        end_eng = content_str_eng.find('Return to:') # 답변의 끝 지점
        answer_html_eng = content_str_eng[start_eng : end_eng]
        answer_eng = BeautifulSoup(answer_html_eng, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기

            
        # 파일 저장
        new_row = pd.DataFrame({"crawling_date":[crawling_date], "big_title_far":[big_title_far],
                                    "Question_FAR": [title_far], "Answer_FAR": [answer_far], "URL_FAR": [url_far],
                                    "Question_ENG": [title_eng], "Answer_ENG": [answer_eng], "URL_ENG": [url_eng]})
        final = pd.concat([final, new_row], ignore_index=True)
            
        driver.back()
        time.sleep(0.1)

        driver.back()
        time.sleep(0.1)

In [13]:
final

,crawling_date,big_title_far,Question_FAR,Answer_FAR,URL_FAR,Question_ENG,Answer_ENG,URL_ENG
0,2025-04-30,مُژدۀ انجیل,آیا شما حیات جاودانی دارید؟,کتاب مقدس به وضوح راه رسیدن به زندگی جاودانه ر...,https://www.gotquestions.org/Farsi/Farsi-Etern...,Got Eternal Life?,The Bible presents a clear path to eternal lif...,https://www.gotquestions.org/eternal-life.html
1,2025-04-30,مُژدۀ انجیل,چگونه می توان آمُرزش الهی را دریافت نمود؟,"اعمال رسولان فصل 13 آیه 38 به ما می گوید که: ""...",https://www.gotquestions.org/Farsi/Farsi-Got-F...,Got Forgiveness?,"Acts 13:38\n declares, ""Therefore, my brothers...",https://www.gotquestions.org/got-forgiveness.html
2,2025-04-30,مُژدۀ انجیل,من یک مسلمان هستم. چرا باید به فکر مسیحی شدن ب...,مردم معمولا دنباله روی دین و مذهب والدین و یا ...,https://www.gotquestions.org/Farsi/Farsi-Musli...,I am a Muslim. Why should I consider becoming ...,People often follow the religion of their pare...,https://www.gotquestions.org/Muslim-Christian....
3,2025-04-30,مُژدۀ انجیل,"آیا چیزی به عنوان ""زندگی پس از مرگ"" وجود دارد؟",اینکه آیا پس از مرگ زندگی وجود دارد یا نه! یه ...,https://www.gotquestions.org/Farsi/Farsi-life-...,Is there life after death?,"Job, speaking out of his despair, asked, “If a...",https://www.gotquestions.org/is-there-life-aft...
4,2025-04-30,مُژدۀ انجیل,پذیرفتن عیسی مسیح به عنوان نجات دهندۀ شخصی به ...,آیا عیسی مسیح را به عنوان نجات دهندۀ شخصی خود ...,https://www.gotquestions.org/Farsi/Farsi-perso...,What does it mean to accept Jesus as your pers...,Have you accepted Jesus Christ as your persona...,https://www.gotquestions.org/personal-Savior.html
...,...,...,...,...,...,...,...,...
1547,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,ما از زندگی پطرس چه درسی می گیریم؟,شمعون پطرس یا کیفا (صخره) (یوحنا فصل 1 آیه 42)...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Peter in the Bible?,"Simon Peter, also known as Cephas (\nJohn 1:42...",https://www.gotquestions.org/life-Peter.html
1548,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,فیلیپ در کتاب مقدس چه کسی بود؟,چهار مرد متفاوت در کتاب مقدس وجود دارند که نام...,https://www.gotquestions.org/Farsi/Farsi-Phili...,Who was Philip in the Bible?,There are four different men named Philip ment...,https://www.gotquestions.org/Philip-in-the-Bib...
1549,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,ما از زندگی استیفان چه درسی می گیریم؟,اعمال رسولان فصل 6 آیه 5، استیفان را مردی با ا...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Stephen in the Bible?,Acts 6:5\n introduces a faithful man of God na...,https://www.gotquestions.org/life-Stephen.html
1550,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,ما از زندگی تیموتائوس چه درسی می گیریم؟,تیموتائوس که دریافت کننده دو نامه عهد جدید به ...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Timothy in the Bible?,"Timothy, the recipient of the two New Testamen...",https://www.gotquestions.org/life-Timothy.html


In [14]:
final.to_excel(f"/Users/haley/Desktop/2025-1/DS1/code/preprocessing/GodQuestions/GodQuestions_raw_Farsi__{crawling_date}.xlsx",index=False)

### Preprocessing_Low

In [ ]:
df_far = pd.read_excel("/Users/haley/Desktop/2025-1/DS1/code/preprocessing/GodQuestions/GodQuestions_raw_Farsi__2025-04-30.xlsx")

In [5]:
df_far

,crawling_date,big_title_far,Question_FAR,Answer_FAR,URL_FAR,Question_ENG,Answer_ENG,URL_ENG
0,2025-04-30,مُژدۀ انجیل,آیا شما حیات جاودانی دارید؟,کتاب مقدس به وضوح راه رسیدن به زندگی جاودانه ر...,https://www.gotquestions.org/Farsi/Farsi-Etern...,Got Eternal Life?,The Bible presents a clear path to eternal lif...,https://www.gotquestions.org/eternal-life.html
1,2025-04-30,مُژدۀ انجیل,چگونه می توان آمُرزش الهی را دریافت نمود؟,"اعمال رسولان فصل 13 آیه 38 به ما می گوید که: ""...",https://www.gotquestions.org/Farsi/Farsi-Got-F...,Got Forgiveness?,"Acts 13:38\n declares, ""Therefore, my brothers...",https://www.gotquestions.org/got-forgiveness.html
2,2025-04-30,مُژدۀ انجیل,من یک مسلمان هستم. چرا باید به فکر مسیحی شدن ب...,مردم معمولا دنباله روی دین و مذهب والدین و یا ...,https://www.gotquestions.org/Farsi/Farsi-Musli...,I am a Muslim. Why should I consider becoming ...,People often follow the religion of their pare...,https://www.gotquestions.org/Muslim-Christian....
3,2025-04-30,مُژدۀ انجیل,"آیا چیزی به عنوان ""زندگی پس از مرگ"" وجود دارد؟",اینکه آیا پس از مرگ زندگی وجود دارد یا نه! یه ...,https://www.gotquestions.org/Farsi/Farsi-life-...,Is there life after death?,"Job, speaking out of his despair, asked, “If a...",https://www.gotquestions.org/is-there-life-aft...
4,2025-04-30,مُژدۀ انجیل,پذیرفتن عیسی مسیح به عنوان نجات دهندۀ شخصی به ...,آیا عیسی مسیح را به عنوان نجات دهندۀ شخصی خود ...,https://www.gotquestions.org/Farsi/Farsi-perso...,What does it mean to accept Jesus as your pers...,Have you accepted Jesus Christ as your persona...,https://www.gotquestions.org/personal-Savior.html
...,...,...,...,...,...,...,...,...
1547,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,ما از زندگی پطرس چه درسی می گیریم؟,شمعون پطرس یا کیفا (صخره) (یوحنا فصل 1 آیه 42)...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Peter in the Bible?,"Simon Peter, also known as Cephas (\nJohn 1:42...",https://www.gotquestions.org/life-Peter.html
1548,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,فیلیپ در کتاب مقدس چه کسی بود؟,چهار مرد متفاوت در کتاب مقدس وجود دارند که نام...,https://www.gotquestions.org/Farsi/Farsi-Phili...,Who was Philip in the Bible?,There are four different men named Philip ment...,https://www.gotquestions.org/Philip-in-the-Bib...
1549,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,ما از زندگی استیفان چه درسی می گیریم؟,اعمال رسولان فصل 6 آیه 5، استیفان را مردی با ا...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Stephen in the Bible?,Acts 6:5\n introduces a faithful man of God na...,https://www.gotquestions.org/life-Stephen.html
1550,2025-04-30,سوالاتی در مورد مردم در کتاب مقدس,ما از زندگی تیموتائوس چه درسی می گیریم؟,تیموتائوس که دریافت کننده دو نامه عهد جدید به ...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Timothy in the Bible?,"Timothy, the recipient of the two New Testamen...",https://www.gotquestions.org/life-Timothy.html


#### ENG Answer Preprocessing 

In [7]:
# 1
# Find : "Have you ~ button below ~""
#df_far["Answer_ENG"].apply(lambda x: re.findall(r"\nHave you[\s\S]*?button below[\s\S]+",x))
df_far["Answer_ENG"].apply(lambda x: re.findall(r"\nHave you[\s\S]*?button below[\s\S]*?", x) if isinstance(x, str) else [])

0       [\nHave you made a decision for Christ because...
1       [\nHave you made a decision for Christ because...
2       [\nHave you trusted Jesus as your Savior and L...
3       [\nHave you made a decision to receive Jesus C...
4       [\nHave you made a decision to receive Jesus C...
                              ...                        
1547                                                   []
1548                                                   []
1549                                                   []
1550                                                   []
1551                                                   []
Name: Answer_ENG, Length: 1552, dtype: object

In [8]:
# Remove : "Have you ~ button below ~""
df_far["Answer_ENG"] = df_far["Answer_ENG"].str.replace(r"\nHave you[\s\S]*?button below[\s\S]+", "", regex=True)

In [10]:
df_far["Answer_ENG"][1]

'Acts 13:38\n declares, "Therefore, my brothers, I want you to know that through Jesus the forgiveness of sins is proclaimed to you."\n\n\n\n\nWhat is forgiveness and why do I need it?\n\nThe word "forgive" means to wipe the slate clean, to pardon, to cancel a debt. When we wrong someone, we seek their forgiveness in order for the relationship to be restored. Forgiveness is not granted because a person deserves to be forgiven. No one deserves to be forgiven. Forgiveness is an act of love, mercy, and grace. Forgiveness is a decision to not hold something against another person, despite what they have done to you.\n\nThe Bible tells us that we are all in need of forgiveness from God. We have all committed sin. \nEcclesiastes 7:20\n proclaims, "There is not a righteous man on earth who does what is right and never sins." \nFirst John 1:8\n says, "If we claim to be without sin, we deceive ourselves and the truth is not in us." All sin is ultimately an act of rebellion against God (\nPsalm 

In [11]:
# 2
# Find : "Related Articles ~"
df_far["Answer_ENG"].apply(lambda x: re.findall(r"\nRelated Articles[\s\S]+",x))

TypeError: expected string or bytes-like object, got 'float'

In [ ]:
# Remove : "Related Articles ~"
df_far["Answer_ENG"] = df_far["Answer_ENG"].str.replace(r"\nRelated Articles[\s\S]+", "", regex=True)

In [ ]:
df_far["Answer_ENG"]

In [ ]:
# 3
# Find : "For Further Study ~"
df_far["Answer_ENG"].apply(lambda x: re.findall(r"\nFor Further Study[\s\S]+",x))

In [ ]:
# Remove : "For Further Study ~"
df_far["Answer_ENG"] = df_far["Answer_ENG"].str.replace(r"\nFor Further Study[\s\S]+", "", regex=True)

In [ ]:
df_far["Answer_ENG"]

In [ ]:
# 4 - unseen duplicate row
# Split : "\n{1,}"